In [1]:
#
# Startup lib and variables
#
import os
import winsound
import random
import cv2
import csv
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Input, Embedding, Dot, Reshape


#Global Variables

image_size = 28
train_dir = 'C:/Users/Public/Downloads/AHDBase_TrainingSet/'
test_dir = 'C:/Users/Public/Downloads/AHDBase_TestingSet/'
train_file = 'train.csv'
test_file = 'test.csv'
log_dir = 'C:/Users/Public/Downloads/Logs/'

data_train = []
data_test = []
training_data = []
testing_data = []
X_train = [] 
y_train =[]
X_test = [] 
y_test =[]



In [5]:
#
# Collection of  of Functions
#

def beep():
    try:
        frequency = 250  # Set Frequency To 2500 Hertz
        duration = 100  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
    except Exception as e:
        pass
    else:
        pass
    finally:
        pass
    

def read_list_of_images():
    try:
        global data_train
        global data_test
       
        #Open Train List
        with open(os.path.join(train_dir, train_file)) as ftrain:  
            train_reader = csv.reader(ftrain)
            next(train_reader) # skip header
            for train_row in train_reader:
                train_row[0] = train_row[0].lower()
                data_train.append(train_row)
        
        #Open Test List
        with open(os.path.join(test_dir, test_file)) as ftest:  
            test_reader = csv.reader(ftest)
            next(test_reader) # skip header
            for test_row in test_reader:
                test_row[0] = test_row[0].lower()
                data_test.append(test_row) 
                
        return data_train, data_test
    except Exception as e:
        print(e) #or pass
    finally:
        ftrain.close()
        ftest.close()
        print('Total Train Items Read: ', len(data_train))
        print('Total Test Items Read: ', len(data_test))
        beep()
    
def load_image_labels(training_count, testing_count):
    try:              
        global training_data
        global testing_data
        global data_train
        global data_test
        
        dim = (image_size, image_size)
        
        for train_row in data_train[:training_count]: #Read up to count rows
            image_path = os.path.join(train_dir, train_row[0])
            #ReShape
            image_read = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) #Gray Scale=smaller array size
            image_sized = cv2.resize(image_read, dim, interpolation = cv2.INTER_CUBIC)
            #Load Image & label
            training_data.append([image_sized, int(train_row[1])])
        
        for testing_row in data_test[:testing_count]: #Read up to count rows
            image_path = os.path.join(test_dir, testing_row[0])
            #Shape
            image_read = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) #Gray Scale=smaller array size
            image_sized = cv2.resize(image_read, dim, interpolation = cv2.INTER_CUBIC)
            #Load Image & label
            testing_data.append([image_sized, int(testing_row[1])])        
        
        random.shuffle(training_data)
        random.shuffle(testing_data)
        
        return training_data, testing_data
    except Exception as e:
        print(e) #or pass
    finally:
        print('Total Train Items Loaded: ', len(training_data))
        print('Total Test Items Loaded: ', len(testing_data))
        beep()
        
def prepare_data_for_NN():
    try:       
        global X_train
        global y_train
        global X_test
        global y_test
        
        for train_features, train_label in training_data:
            X_train.append(train_features)
            y_train.append(train_label)
        X_train = np.array(X_train).reshape(-1, image_size ,image_size, 1) #1 for gray, 3 for color
        y_train = np.asarray(y_train)
        
        for test_features, test_label in testing_data:
            X_test.append(test_features)
            y_test.append(test_label)
        X_test = np.array(X_test).reshape(-1, image_size ,image_size, 1) #1 for gray, 3 for color
        y_test = np.asarray(y_test)
        
        
        return X_train, y_train, X_test, y_test
    except Exception as e:
        print(e) #or pass
    finally:
        print('Total X-train Items Processed: ', len(X_train))
        print('Total y_train Items Processed: ', len(y_train))
        print('Total X-Test Items Processed: ', len(X_test))
        print('Total y_test Items Processed: ', len(y_test))        
        beep()
        
        
def save_pickle():
    try:
        #save Training data
        pickle_path = os.path.join(train_dir,'X_train.pickle')
        pickle_out =  open(pickle_path,'wb')
        pickle.dump(X_train, pickle_out)
        pickle_out.close()

        pickle_path = os.path.join(train_dir,'y_train.pickle')
        pickle_out =  open(pickle_path,'wb')
        pickle.dump(y_train, pickle_out)
        pickle_out.close()
        
        #save Testing Data
        pickle_path = os.path.join(test_dir,'X_test.pickle')
        pickle_out =  open(pickle_path,'wb')
        pickle.dump(X_test, pickle_out)
        pickle_out.close()

        pickle_path = os.path.join(test_dir,'y_test.pickle')
        pickle_out =  open(pickle_path,'wb')
        pickle.dump(y_test, pickle_out)
        pickle_out.close()
        
    except Exception as e:
        print(e) #or pass
    finally:
        print('Pickles Saved.')       
        beep()

        
def load_pickle():
    try:
        global X_train
        global y_train
        global X_test
        global y_test   
        
        #Load Traiing Pickle
        pickle_path = os.path.join(train_dir, 'X_train.pickle')
        with open(pickle_path, 'rb') as X_train_file:
            X_train = pickle.load(X_train_file)
            
        pickle_path = os.path.join(train_dir,'y_train.pickle')
        with open(pickle_path, 'rb') as y_train_file:
            y_train = pickle.load(y_train_file)  
            
        #Load Testing Pickle
        pickle_path = os.path.join(test_dir, 'X_test.pickle')
        with open(pickle_path, 'rb') as X_test_file:
            X_test = pickle.load(X_test_file)
            
        pickle_path = os.path.join(test_dir,'y_test.pickle')
        with open(pickle_path, 'rb') as y_test_file:
            y_test = pickle.load(y_test_file)             
            
        return X_train, y_train, X_test, y_test
    except Exception as e:
        print(e) #or pass
    finally:
        X_train_file.close()
        y_train_file.close()
        X_test_file.close()
        y_test_file.close()
        print('X_train pickels read: ', len(X_train))
        print('y_train pickels read: ', len(y_train))
        print('X_test pickels read: ', len(X_test))
        print('y_test pickels read: ', len(y_test))
        beep()



In [ ]:
read_list_of_images() 

In [ ]:
load_image_labels(training_count = len(data_train), testing_count = len(data_test))

In [ ]:
prepare_data_for_NN()

In [ ]:
save_pickle()

In [6]:
load_pickle()

X_train pickels read:  60000
y_train pickels read:  60000
X_test pickels read:  10000
y_test pickels read:  10000


(array([[[[253],
          [255],
          [255],
          ...,
          [255],
          [255],
          [255]],
 
         [[255],
          [255],
          [153],
          ...,
          [255],
          [255],
          [255]],
 
         [[  0],
          [  0],
          [  0],
          ...,
          [255],
          [255],
          [255]],
 
         ...,
 
         [[255],
          [255],
          [255],
          ...,
          [158],
          [248],
          [255]],
 
         [[255],
          [255],
          [255],
          ...,
          [175],
          [255],
          [255]],
 
         [[255],
          [255],
          [255],
          ...,
          [  0],
          [  0],
          [  0]]],
 
 
        [[[255],
          [255],
          [255],
          ...,
          [255],
          [255],
          [255]],
 
         [[255],
          [255],
          [120],
          ...,
          [255],
          [255],
          [255]],
 
         [[255],
    

In [7]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir = 'SimLogs')

In [8]:
#Model One
# See: https://elitedatascience.com/keras-tutorial-deep-learning-in-python
#

X_train = X_train/255.0

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape= X_test.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 128, epochs = 10, validation_split = 0.1, callbacks = [tensorboard])
model.summary()

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 8s 157us/sample - loss: 0.3411 - accuracy: 0.8879 - val_loss: 0.0859 - val_accuracy: 0.9720
Epoch 2/10
54000/54000 [==============================] - 4s 78us/sample - loss: 0.0726 - accuracy: 0.9779 - val_loss: 0.0403 - val_accuracy: 0.9885
Epoch 3/10
54000/54000 [==============================] - 4s 79us/sample - loss: 0.0512 - accuracy: 0.9844 - val_loss: 0.0379 - val_accuracy: 0.9882
Epoch 4/10
54000/54000 [==============================] - 4s 80us/sample - loss: 0.0402 - accuracy: 0.9873 - val_loss: 0.0280 - val_accuracy: 0.9907
Epoch 5/10
54000/54000 [==============================] - 4s 79us/sample - loss: 0.0370 - accuracy: 0.9884 - val_loss: 0.0281 - val_accuracy: 0.9905
Epoch 6/10
54000/54000 [==============================] - 4s 77us/sample - loss: 0.0333 - accuracy: 0.9894 - val_loss: 0.0237 - val_accuracy: 0.9925
Epoch 7/10
54000/54000 [==============================] 

In [9]:
#Score Model
X_test = X_test/255.0
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

[0.033528009092784486, 0.9905]
